In [1]:
# import libraries
import pandas as pd
import math

In [2]:
# reading the dataframe
df = pd.read_csv("golf_df.csv")
print(df.shape)
df.head()

(14, 5)


,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [3]:
# train test split
train = df[:(math.ceil(df.shape[0]*0.8))]
test = df[(math.ceil(df.shape[0]*0.8)):]

X_train = train.drop(['Play'],axis=1)
y_train = train['Play']

X_test = test.drop(['Play'],axis=1)
y_test = test['Play']

print(X_train.shape,X_test.shape)

(12, 4) (2, 4)


In [4]:
def learning(df,X,y,test):
    #computing prior probability
    prior_prob = {}
    for var in y.unique():
        temp = y[ y == var].count()/y.count()
        prior_prob[var] = temp
    
    # learning phase
    learning = {}
    target = y.name
    for col in df.columns:
        if col != y.name:
            temp = pd.DataFrame(df.groupby([col,target]).count()/df.groupby([target]).count()).drop(col,axis=1).reset_index()
            temp = temp[temp.columns[:3]].rename(columns={temp.columns[2]:"Value"})
            learning[col] = temp
            
    # prediction
    class_prob = {}
    for var in y.unique():
        product = 1
        for col in X.columns:
            try:
                temp = learning[col].loc[(learning[col][col] == test[col])&(learning[col][target] == var)]['Value'].values[0]
            except Exception as e:
                temp = 0
            product = product * temp
        product = product * prior_prob[var]
        class_prob[product] = var
    
    print(class_prob)
    max_ = max(class_prob.keys())
    return class_prob[max_]

In [5]:
# predictions
y_pred = []
for index, row in X_test.iterrows():
    y_pred.append(learning(df, X_train ,y_train ,row.to_dict()))

{0.0: 'no', 0.029263831732967528: 'yes'}
{0.025600000000000005: 'no', 0.010973936899862823: 'yes'}


In [6]:
y_pred

['yes', 'no']

In [8]:
y_test.tolist()

['yes', 'no']